In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

# Functions for web scrapping columns

In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


# To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the List of Falcon 9 and Falcon Heavy launches Wikipage updated on 9th June 2021

In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

# TASK 1: Request the Falcon9 Launch Wiki page from its URL

In [4]:
# use requests.get() method with the provided static_url and headers
# assign the response to a object

# Hacer la petición con headers
response = requests.get(static_url, headers=headers)

# Verificar que la página se cargó correctamente
if response.status_code == 200:
    print("Conexión exitosa!")
    content = response.text
else:
    print("Error:", response.status_code)

Conexión exitosa!


In [5]:
soup = BeautifulSoup(content, 'html')
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

# TASK 2: Extract all column/variable names from the HTML table header

In [6]:
html_tables = soup.find_all('table')
len(html_tables)

25

In [7]:
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

In [8]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the No-tabn-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
for col in first_launch_table.find_all('th'):
    name = extract_column_from_header(col)
    if name is not None and len(name) > 0:
        column_names.append(name)
column_names

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

# TASK 3: Create a data frame by parsing the launch HTML tables
## We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe

In [9]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

# Next, we just need to fill up the launch_dict with launch records extracted from table rows.

In [10]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv = row[1].get_text(strip=True)
            launch_dict['Version Booster'].append(bv)
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].get_text(strip=True)
            launch_dict['Launch site'].append(launch_site)
            print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].get_text(strip=True)
            launch_dict['Payload'].append(payload)
            print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            print(payload_mass)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].get_text(strip=True)
            launch_dict['Orbit'].append(orbit)
            print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].get_text(strip=True)
            launch_dict['Customer'].append(customer)
            print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            print(booster_landing)
            

1
4 June 2010
18:45
F9 v1.07B0003.18
CCAFS,SLC-40
Dragon Spacecraft Qualification Unit
0
LEO
SpaceX
Success

Failure
2
8 December 2010
15:43
F9 v1.07B0004.18
CCAFS,SLC-40
Dragondemo flight C1(Dragon C101)
0
LEO(ISS)
NASA(COTS)NRO
Success
Failure
3
22 May 2012
07:44
F9 v1.07B0005.18
CCAFS,SLC-40
Dragondemo flight C2+[18](Dragon C102)
525 kg
LEO(ISS)
NASA(COTS)
Success
No attempt

4
8 October 2012
00:35
F9 v1.07B0006.18
CCAFS,SLC-40
SpaceX CRS-1[22](Dragon C103)
4,700 kg
LEO(ISS)
NASA(CRS)
Success

No attempt
5
1 March 2013
15:10
F9 v1.07B0007.18
CCAFS,SLC-40
SpaceX CRS-2[22](Dragon C104)
4,877 kg
LEO(ISS)
NASA(CRS)
Success

No attempt

6
29 September 2013
16:00
F9 v1.17B10038
VAFB,SLC-4E
CASSIOPE[22][31]
500 kg
Polar orbitLEO
MDA
Success
Uncontrolled
7
3 December 2013
22:41
F9 v1.1B1004
CCAFS,SLC-40
SES-8[22][35][36]
3,170 kg
GTO
SES
Success
No attempt
8
6 January 2014
22:06
F9 v1.1
CCAFS,SLC-40
Thaicom 6[22]
3,325 kg
GTO
Thaicom
Success
No attempt
9
18 April 2014
19:25
F9 v1.1
Cape Can

In [11]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,"CCAFS,SLC-40",Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,"CCAFS,SLC-40",Dragondemo flight C1(Dragon C101),0,LEO(ISS),NASA(COTS)NRO,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,"CCAFS,SLC-40",Dragondemo flight C2+[18](Dragon C102),525 kg,LEO(ISS),NASA(COTS),Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,"CCAFS,SLC-40",SpaceX CRS-1[22](Dragon C103),"4,700 kg",LEO(ISS),NASA(CRS),Success\n,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,"CCAFS,SLC-40",SpaceX CRS-2[22](Dragon C104),"4,877 kg",LEO(ISS),NASA(CRS),Success\n,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


In [12]:
df.to_csv('C:/Users/Nancy Sorroza Rojas/Desktop/Pool/Proyectos Data Science/Space X practice/data/spacex_web_scraped.csv', index=False)